# Representational drift across sensory discrimination learning:
- 230212 Representational drift during learning 01.pptx
- Questions:
    1. Are session-to-session touch response and tuning random?
    - Compare with shuffling
    - Mean of cosine similarity, after binarizing touch and tuning.
    2. What drives neurons to be [recuirted to / removed from] touch and tuning?
    - Previous session response characteristic
        - Before or after answer
    - Previous session activity or noise correlation to the 'core' neurons.
        - Before or after answer
    - How does classifier perform across sessions? (trained in one session and applied to another)


In [4]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import os, glob, h5py
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns 
from tqdm import notebook
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib as mpl
import socket
%matplotlib inline

# h5_dir_1 = Path('F:/')
# h5_dir_2 = Path('D:/')
mice =          [25,    27,   30,   36,     37,     38,     39,     41,     52,     53,     54,     56]
expert_mice = [25, 27, 30, 36, 39, 52]
ref_sessions = [4,3,3,1,1,3]
exp_sessions = [19,10,21,17,23,21]
# refSessions =   [4,     3,    3,    1,      7,      2,      1,      3,      3,      3,      3,      3]
# expSessions =   [19,    10,   21,   17,     0,      0,      23,     0,      21,     0,      0,      0]
# zoom =          [2,     2,    2,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7]
image_rate_list = [30.8359, 30.8359, 30.8359, 30.8359, 
                    24.2892, 24.2892, 24.2892, 24.2892,
                    30.8359, 30.8359, 30.8359, 30.8359] # from .mat file (sbx)
image_size_list = [[512,796], [512,796], [512,796], [512,796], 
                    [650,796], [650,796], [650,796], [650,796], 
                    [512,796], [512,796], [512,796], [512,796]] # from .mat file (sbx)
planes = range(1,9)

if socket.gethostname() == 'OSXLTR6HX40':
    save_dir_base = Path('/Users/jinho.kim/Dropbox/Works/Projects/2020 Neural stretching in S1/Results')
else:
    save_dir_base = Path(r'C:\Users\shires\Dropbox\Works\Projects\2020 Neural stretching in S1\Results')


## Control the p-value with the # of samples.
- p-values are affected by the # of samples
- Need to control the # of samples.
- Check the minimum # of samples per session, for 135 and 45 touch trials.
- Across all mice.
- Keep the same # of samples.
- Bootstrap 100 times and take the mean. Both for the p-value and response amplitude.
- (Control) Compare the bootstrapped mean amplitude with the whole-sample response amplitude.


In [5]:
# In one plane
mouse = 25
plane = 3
# Check the # of touch trials
load_fn = f'JK{mouse:03d}_plane{plane}_touch_response_crude.pkl'
tr_crude = pd.read_pickle(save_dir_base / load_fn)



In [6]:
#



,session,num_roi,correct_rate,num_answer,num_touch_trials,pre_pole_len,angle,correct,num_touch_pre_answer,num_touch_post_answer,p_pre_answer,p_post_answer,effect_size_pre_answer,effect_size_post_answer,effect_size_post_pre,pre_pole_activity,pre_answer_response,post_answer_response,post_pole_activity
0,1,254,0.253333,87,86,"[7.0, 6.0, nan, nan, 5.0, 5.0, 5.0, 6.0, nan, ...","[135.0, 45.0, nan, nan, 135.0, 135.0, 45.0, 45...","[1.0, 0.0, nan, nan, 1.0, 1.0, 0.0, 1.0, nan, ...","[18.0, 11.0, nan, nan, 10.0, 2.0, 31.0, 8.0, n...","[9.0, 0.0, nan, nan, 6.0, 1.0, 0.0, 3.0, nan, ...","[1.5862196740222777e-13, 6.792500918017778e-28...","[7.292040623467817e-05, 7.2517312313589e-06, 5...","[0.10000942013509356, 0.16933925657889204, 0.1...","[0.08903506524270265, 0.28088910499496794, 0.1...","[-0.012811894380911326, 0.10291229875438149, -...","[[0.18001770973205566, -0.07637003809213638, 0...","[[0.020445918664336205, 0.2431962490081787, 0....","[[-0.003148887772113085, 0.1778194159269333, -...","[[-0.10715257376432419, -0.165530726313591, -0..."
1,2,254,0.366667,253,243,"[nan, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 5.0, ...","[nan, 45.0, 135.0, 135.0, 135.0, 45.0, 135.0, ...","[nan, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[nan, 17.0, 2.0, 3.0, 10.0, 3.0, 4.0, 32.0, 5....","[nan, 10.0, 8.0, 7.0, 10.0, 0.0, 8.0, 0.0, 9.0...","[7.2657073003020405e-09, 1.7633712623268079e-4...","[1.3554871462737156e-07, 4.4430712938880164e-0...","[0.07701880192632823, 0.14324955935198924, 0.0...","[0.08807738557463216, 0.19289689799770712, 0.0...","[0.011215188469776132, 0.036557948380835696, -...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2,3,254,0.410000,173,167,"[nan, 5.0, 5.0, 6.0, 5.0, nan, nan, 5.0, 5.0, ...","[nan, 45.0, 135.0, 45.0, 45.0, nan, nan, 135.0...","[nan, 0.0, 0.0, 1.0, 0.0, nan, nan, 1.0, 1.0, ...","[nan, 5.0, 16.0, 7.0, 3.0, nan, nan, 12.0, 12....","[nan, 0.0, 0.0, 12.0, 0.0, nan, nan, 11.0, 8.0...","[2.3884406620779246e-11, 4.198059479711445e-15...","[1.0601825073298871e-05, 1.3484975722653315e-0...","[0.07923032937342105, 0.12345507757369294, 0.0...","[0.06586363391065395, 0.15877920120845285, 0.0...","[-0.0016587760368296316, 0.0074281183777112555...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
3,5,254,0.392344,174,164,"[nan, nan, nan, nan, 5.0, 5.0, 5.0, 6.0, 5.0, ...","[nan, nan, nan, nan, 45.0, 135.0, 45.0, 45.0, ...","[nan, nan, nan, nan, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[nan, nan, nan, nan, 1.0, 9.0, 2.0, 1.0, 4.0, ...","[nan, nan, nan, nan, 0.0, 2.0, 0.0, 0.0, 7.0, ...","[0.0004432515760979102, 1.4405472571882788e-17...","[0.024523793757963323, 1.3509986827617543e-09,...","[0.06152246924447306, 0.13880669037462212, 0.0...","[0.1020765417641087, 0.11034043604886393, 0.11...","[0.07024138122891171, -0.04915292988709338, 0....","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
4,6,254,0.453333,139,124,"[8.0, 5.0, nan, 5.0, 5.0, 5.0, 5.0, 5.0, nan, ...","[45.0, 135.0, nan, 135.0, 135.0, 135.0, 135.0,...","[0.0, 1.0, nan, 0.0, 0.0, 1.0, 1.0, 0.0, nan, ...","[15.0, 11.0, nan, 9.0, 7.0, 12.0, 2.0, 2.0, na...","[0.0, 6.0, nan, 0.0, 0.0, 6.0, 2.0, 0.0, nan, ...","[8.698207068656364e-13, 1.6406792872223835e-15...","[4.158431582891248e-07, 9.482468958923123e-12,...","[0.09254972540015766, 0.14330084188629172, -0....","[0.11486500588489434, 0.17654656908916877, -0....","[0.015105706540233424, 0.032287261121131575, 0...","[[-0.15665994584560394, -0.13569359481334686, ...","[[0.02142762765288353, -0.009046238847076893, ...","[[nan, nan, nan, nan, nan, nan, nan, na

## Which methods best describe representational drift?
- Cosine similarity, correlation, Jaccard distance (I/U)
    - Events are z-scored, so cosine similarity over events is actually the same as correlation.
        - However, when these are calculated over response amplitude or touch responsiveness p-value, then those can differ.
            - Cosine similarity cares about shift, while correlation does not.
        - Either of these are without binarizing. 
        - Thresholding can be applied.
    - Jaccard distance requires binarization.
- Controls:
    - odd/even trials (after removing non-touch and no-answer trials)
    - first/second half
- Need to read first.
    - Most used correlation.
    - Aitken et al., used angle between vectors.
    - Sadeh & Clopath showed results that support thresholding or binarizing.
- Let's try all and see which makes more sense.

In [3]:
# In one example volume
mouse = 25
volume = 0 # 0 or 1 (0: upper, 1: lower)
if volume == 0:
    planes = range(1,5)
elif volume == 1:
    planes = range(5,9)
else:
    raise ValueError('volume should be 0 or 1')

# Load data
tr_crude_list = []
for plane in planes:
    load_fn = f'JK{mouse:03d}_plane{plane}_touch_response_crude.pkl'
    tr_crude = pd.read_pickle(save_dir_base / load_fn)
    tr_crude_list.append(tr_crude)


In [8]:
row = tr_crude_list[0].iloc[0]
# print length of each value in the row
for key in row.keys():
    try:
        print(f'{key}: {len(row[key])}')
    except:
        print(f'{key}: {row[key]}')


session: 1
num_roi: 155
correct_rate: 0.25333333333333335
num_answer: 87
num_touch_trials: 86
pre_pole_len: 150
angle: 150
correct: 150
num_touch_pre_answer: 150
num_touch_post_answer: 150
p_pre_answer: 155
p_post_answer: 155
effect_size_pre_answer: 155
effect_size_post_answer: 155
effect_size_post_pre: 155
pre_pole_activity: 150
pre_answer_response: 150
post_answer_response: 150
post_pole_activity: 150


In [9]:
# Getting odd/even trials
ind_45_trials = np.where(row.angle==45)[0]
ind_135_trials = np.where(row.angle==135)[0]

odd_45_trials = ind_45_trials[::2]
even_45_trials = ind_45_trials[1::2]
odd_135_trials = ind_135_trials[::2]
even_135_trials = ind_135_trials[1::2]
odd_trials = np.concatenate([odd_45_trials, odd_135_trials])
even_trials = np.concatenate([even_45_trials, even_135_trials])
print(len(odd_trials), len(even_trials))

44 42


In [10]:
# Getting first/second half trials
# After dividing into 45 and 135 trials
ind_45_first_half = ind_45_trials[:len(ind_45_trials)//2]
ind_45_second_half = ind_45_trials[len(ind_45_trials)//2:]
ind_135_first_half = ind_135_trials[:len(ind_135_trials)//2]
ind_135_second_half = ind_135_trials[len(ind_135_trials)//2:]
first_half = np.concatenate([ind_45_first_half, ind_135_first_half])
second_half = np.concatenate([ind_45_second_half, ind_135_second_half])
print(len(first_half), len(second_half))

42 44


In [ ]:
# Correlation using all cells
# Correlation of -log(p) values
# Correlation of response amplitudes (z-scored)

# Between first and second sessions
 